In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
import os

# Specify the path to the ZIP file containing the images
zip_file_path = '/content/drive/MyDrive/Photos_Walking Audit-20230701T115400Z-001.zip'

# Specify the directory where you want to extract the images
extracted_images_dir = '/content/extracted_images'

# Extract the images from the ZIP file to the specified directory
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_images_dir)

# Get the path to the extracted image directory
image_dir_path = os.path.abspath(extracted_images_dir)

# Print the path to the image directory
print("Path to the extracted image directory:", image_dir_path)


Path to the extracted image directory: /content/extracted_images


In [3]:
import pandas as pd

# Read the Excel file
data = pd.read_excel('/content/Issues_Collected_category and issue.xlsx')


In [4]:
image_file_names = data['Photo']
l={}
for i in range(0,len(image_file_names)):
  l[image_file_names[i]] = data.iloc[i]['Issue type']

In [5]:
from sklearn.preprocessing import OneHotEncoder
label_encoder = OneHotEncoder()
l1 = list(set(l.values()))
print(l1)


# Reshape the label values to a 2D array-like shape
l1_2d = [[value] for value in l1]

# Fit and transform the reshaped labels
labels_encoded = label_encoder.fit_transform(l1_2d).toarray()

# Create a dictionary to map the encoded labels to their original values
encoded_labels = {}
for i, label in enumerate(l1):
    encoded_labels[label] = labels_encoded[i]

print(encoded_labels)
print(encoded_labels.keys())



['Encroachment by vendor or shop spillover', 'Transformers', 'Junction boxes', 'Silt accumulation in drains', 'Dark zone', 'Broken footpath', 'Garbage vulnerable point', 'Encroachment by planters', 'Encroachment by parking', 'Litter on street (Commercial)', 'Construction debris', 'Yellow spots', 'Silt accumulation on road', 'No footpath', 'Overgrown weeds', 'Low hanging wires', 'Level difference']
{'Encroachment by vendor or shop spillover': array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'Transformers': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]), 'Junction boxes': array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'Silt accumulation in drains': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]), 'Dark zone': array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 'Broken footpath': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [6]:
import cv2
import numpy as np

# Define the image dimensions
img_width, img_height = 224, 224

# Create empty lists to store the preprocessed images and labels
preprocessed_images = []
preprocessed_labels = []
i=0
# Iterate over the image file names
for file_name in image_file_names:
    # print(file_name)
    # Load the image using OpenCV or PIL
    file_path = os.path.join('/content/extracted_images/Photos_Walking Audit/', file_name)
    img = cv2.imread('/content/extracted_images/Photos_Walking Audit/' + file_name)  # Update the path accordingly
    # print(img)

    # Preprocess the image (resize, normalize, etc.)
    # try:
    if img is not None:
      img = cv2.resize(img, (img_width, img_height))
      img = img.astype('float32') / 255.0

    # Add the preprocessed image and corresponding label to the lists
      preprocessed_images.append(img)
      preprocessed_labels.append(encoded_labels[l[file_name]])



# Convert the lists to NumPy arrays
preprocessed_images = np.array(preprocessed_images)
preprocessed_labels = np.array(preprocessed_labels)



In [7]:
from sklearn.model_selection import train_test_split
# Split the dataset into train, validation, and test sets
X_train, X_val_test, y_train, y_val_test = train_test_split(preprocessed_images,preprocessed_labels, test_size=0.1, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.2, random_state=42)
print(len(X_train))


print(y_train[0])


540
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]


In [8]:
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Conv2D, Flatten, BatchNormalization, Dense, MaxPooling2D, Activation,Dropout

In [14]:
num_classes = 17
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

# # Load pre-trained VGG16 model (excluding the fully connected layers)
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# # Freeze the base model layers to prevent them from being updated during training
# base_model.trainable = False

# # Create a new model
# model = Sequential()

# # Add the pre-trained VGG16 base model to the new model
# model.add(base_model)

# # Add custom fully connected layers on top of the pre-trained model
# model.add(Flatten())
# model.add()
# model.add(Dense(256, activation='relu'))
# model.add(Dense(512, activation='relu'))
# model.add(Dense(num_classes, activation='softmax'))


# Load pre-trained VGG16 model (excluding the fully connected layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers to prevent them from being updated during training
base_model.trainable = False

# Create a new model
model = Sequential()

# Add the pre-trained VGG16 base model to the new model
model.add(base_model)

# Add additional Conv2D layers
# model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))

# Add custom fully connected layers on top of the pre-trained model
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))


# Compile the model
model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['accuracy'])

# Print the model summary
model.summary()

# Train the model
model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val))

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


58889256/58889256 [==============================] - 1s 0us/step
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 conv2d_20 (Conv2D)          (None, 7, 7, 128)         589952    
                                                                 
 conv2d_21 (Conv2D)          (None, 7, 7, 256)         295168    
                                                                 
 conv2d_22 (Conv2D)          (None, 7, 7, 512)         1180160   
                                                                 
 flatten_5 (Flatten)         (None, 25088)             0         
                                                                 
 dense_20 (Dense)            (None, 256)               6422784   
                                                       

In [15]:
from tensorflow.keras.models import load_model
model.save('/content/drive/MyDrive/my_models/final_pavement.h5')